### 0. Thoughts and ideas 

#### 0.1 General

#### 0.2 About the datasets

##### Brainstorming - 18.09.2024

In [ ]:
# IMPORTS

import numpy as np
import pandas as pd
import xgboost as xgb

### 1. Data 

#### 1.1 Load data into dataframes

In [5]:



ais_test_data_path = '../../Project materials/ais_test.csv'
ais_train_data_path = '../../Project materials/ais_train.csv'

ais_data_test = pd.read_csv(ais_test_data_path)
ais_data_train = pd.read_csv(ais_train_data_path, sep='|')

ais_data_test.head()
ais_data_train.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,61e9f3d4b937134a3c4bff1f,634c4de270937fc01c3a7689
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,61e9f436b937134a3c4c0131,61d3847bb7b7526e1adf3d19
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,61e9f3b4b937134a3c4bfe77,61d36f770a1807568ff9a126
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,61e9f41bb937134a3c4c0087,634c4de270937fc01c3a74f3


In [11]:
#Create dataframes for each ship-id:

ship_train_groups = ais_data_train.groupby('vesselId')
ship_train_dataframes = {ship_id: group for ship_id, group in ship_train_groups}

print(ship_train_dataframes)


688
688


#### 1.2 Split data into X and Y

In [ ]:
ais_data_train_X = ais_data_train[]

### 2. Try to create predictions using simple models:

#### 2.1 XG-boost

In [ ]:
xgb_simple = xgb.XGBClassifier()